## Overview ##

This is a starter notebook inspired by last year's [Logistic Regression on Tournament Seeds by Kasper P. Lauritzen](https://www.kaggle.com/kplauritzen/notebookde27b18258?scriptVersionId=804590) starter kernel. It creates a basic logistic regression model based on the seed differences between teams. 

Note that the predictions for Stage 1's sample submissions file are already based on known outcomes, and the Tourney data this model is trained on includes that data. For Stage 2, you will be predicting future outcomes based on the teams selected for the tournament on March 11.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('mode.chained_assignment', None)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

data_dir = '../data/'

## Load the training data ##
I'm building off of the starter notebook by including DetailedResults for a "past 10 game" average, and season ending ELO ratings from Liam Kirwin

In [3]:
df_average_data = pd.read_csv(data_dir + 'rolling_average_data10.csv')
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_elo_ratings = pd.read_csv(data_dir + 'season_elos.csv')

In [4]:
df_average_data.tail()

,Season,TeamID,ScoreAvg,AstAvg,TOAvg,StlAvg,BlkAvg,ORbAvg,DRbAvg,PFAvg,FGPAvg,3PPAvg,FTPAvg
3113,2018,3460,70.2,12.7,12.4,9.8,3.7,16.1,23.5,15.8,38.746491,29.170725,74.875707
3114,2018,3461,64.8,16.1,11.1,4.1,2.2,8.2,25.0,13.5,45.525980,42.761701,75.642790
3115,2018,3462,52.3,9.7,14.8,6.2,1.7,8.8,21.0,13.9,37.621236,23.627575,62.248416
3116,2018,3463,62.0,11.5,12.9,8.7,3.0,10.2,25.8,14.2,38.713553,25.953182,65.352922
3117,2018,3464,67.0,15.0,13.7,4.8,3.8,9.7,26.6,16.3,40.325619,36.793020,75.067294


In [5]:
df_elo_ratings = df_elo_ratings.rename(columns={'team_id':'WTeamID', 'season': 'Season'})
df_elo_ratings.head()

,Season,season_elo,WTeamID
0,2014,1466.742705,3101
1,2015,1387.712167,3101
2,2016,1548.082286,3101
3,2017,1561.905894,3101
4,2018,1409.454524,3101


First, we'll simplify the datasets to remove the columns we won't be using and convert the seedings to the needed format (stripping the regional abbreviation in front of the seed).

In [6]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

,Season,TeamID,seed_int
0,1998,3330,1
1,1998,3163,2
2,1998,3112,3
3,1998,3301,4
4,1998,3272,5


In [7]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208


## Merge seed for each team ##
Merge the Seeds and ELO ratings with their corresponding TeamIDs in the compact results dataframe.

In [8]:
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff
0,1998,3104,3422,2,15,-13
1,1998,3112,3365,3,14,-11
2,1998,3163,3193,2,15,-13
3,1998,3198,3266,7,10,-3
4,1998,3203,3208,10,7,3


In [9]:
df_concat = pd.merge(left=df_concat, right=df_elo_ratings, how='left', on=['Season', 'WTeamID'])

In [10]:
df_concat = df_concat.rename(columns={'season_elo': 'WTeamELO'})
df_elo_ratings = df_elo_ratings.rename(columns={'WTeamID': 'LTeamID'})

In [11]:
df_concat = pd.merge(left=df_concat, right=df_elo_ratings, how='left', on=['Season', 'LTeamID'])

In [12]:
df_concat = df_concat.rename(columns={'season_elo': 'LTeamELO'})

In [13]:
df_concat.tail()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff,WTeamELO,LTeamELO
1255,2017,3163,3332,1,10,-9,2565.874293,1811.861657
1256,2017,3376,3199,1,3,-2,2282.594911,2088.322098
1257,2017,3280,3163,2,1,1,2064.219596,2565.874293
1258,2017,3376,3390,1,2,-1,2282.594911,2114.860666
1259,2017,3376,3280,1,2,-1,2282.594911,2064.219596


In [14]:
df_concat.Season.unique()

array([1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008,
       2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [15]:
df_average_data.Season.unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [16]:
# Also drop the rows for which we don't have rolling averages
df_concat = df_concat[df_concat['Season'] > 2009]

In [17]:
df_concat.tail()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff,WTeamELO,LTeamELO
1255,2017,3163,3332,1,10,-9,2565.874293,1811.861657
1256,2017,3376,3199,1,3,-2,2282.594911,2088.322098
1257,2017,3280,3163,2,1,1,2064.219596,2565.874293
1258,2017,3376,3390,1,2,-1,2282.594911,2114.860666
1259,2017,3376,3280,1,2,-1,2282.594911,2064.219596


In [18]:
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff,WTeamELO,LTeamELO
756,2010,3124,3201,4,13,-9,1998.425733,1759.961121
757,2010,3173,3395,8,9,-1,1791.393663,1824.234227
758,2010,3181,3214,2,15,-13,2178.977412,1473.166913
759,2010,3199,3256,3,14,-11,2014.441093,1679.172624
760,2010,3207,3265,5,12,-7,1836.022146,1800.448677


In [19]:
df_average_data.tail()

,Season,TeamID,ScoreAvg,AstAvg,TOAvg,StlAvg,BlkAvg,ORbAvg,DRbAvg,PFAvg,FGPAvg,3PPAvg,FTPAvg
3113,2018,3460,70.2,12.7,12.4,9.8,3.7,16.1,23.5,15.8,38.746491,29.170725,74.875707
3114,2018,3461,64.8,16.1,11.1,4.1,2.2,8.2,25.0,13.5,45.525980,42.761701,75.642790
3115,2018,3462,52.3,9.7,14.8,6.2,1.7,8.8,21.0,13.9,37.621236,23.627575,62.248416
3116,2018,3463,62.0,11.5,12.9,8.7,3.0,10.2,25.8,14.2,38.713553,25.953182,65.352922
3117,2018,3464,67.0,15.0,13.7,4.8,3.8,9.7,26.6,16.3,40.325619,36.793020,75.067294


In [20]:
df_average_data[df_average_data.isnull().any(axis=1)]

,Season,TeamID,ScoreAvg,AstAvg,TOAvg,StlAvg,BlkAvg,ORbAvg,DRbAvg,PFAvg,FGPAvg,3PPAvg,FTPAvg
13,2010,3116,61.0,11.6,15.2,8.6,4.7,11.2,22.1,16.5,38.872081,32.646624,NaN
88,2010,3196,57.6,12.3,17.9,7.3,2.7,15.3,20.5,16.5,35.233808,28.827669,NaN
610,2011,3385,61.6,11.6,15.3,7.9,3.6,11.3,22.2,14.6,40.841830,NaN,71.138948
661,2011,3437,52.9,13.7,13.1,6.6,2.5,6.3,22.1,14.4,37.984198,29.711462,NaN
754,2012,3177,71.3,18.3,15.6,7.6,3.4,11.0,25.3,14.1,46.295029,31.068259,NaN
775,2012,3198,62.4,12.7,15.5,7.9,4.1,13.6,24.0,14.4,40.774181,29.131789,NaN
915,2012,3348,46.6,8.6,17.9,7.0,1.1,9.5,19.1,14.3,36.592149,NaN,72.705073
1112,2013,3193,49.8,12.3,14.0,4.9,2.6,11.4,23.8,12.4,36.537347,24.933937,NaN
1191,2013,3276,57.4,12.6,13.1,5.9,3.2,10.6,22.7,11.0,40.829764,28.880063,NaN
1214,2013,3300,40.6,5.9,20.1,6.5,1.3,13.1,23.9,10.9,30.343092,NaN,71.094323


In [21]:
# identify columns with NaN (dirty free throw data) and "normalize" to 70%
#df_average_data[df_average_data.isnull().any(axis=1)]
#values= {'FTPAvg': 70.0}
#df_average_data = df_average_data.fillna(value=values)

In [22]:
len(df_average_data)

3118

In [26]:
df_average_data = df_average_data.dropna(axis=0, how='any')

In [27]:
df_average_data[df_average_data.isnull().any(axis=1)]

,Season,TeamID,ScoreAvg,AstAvg,TOAvg,StlAvg,BlkAvg,ORbAvg,DRbAvg,PFAvg,FGPAvg,3PPAvg,FTPAvg


In [28]:
len(df_average_data)

3091

Now we'll create a vector that contains rolling average stats along with the ELO ratings for both winning and losing teams. 
Also want to randomize whether or not Team 1 or Team 2 is first in the X vector

In [31]:
df_average_data.head()

,Season,TeamID,ScoreAvg,AstAvg,TOAvg,StlAvg,BlkAvg,ORbAvg,DRbAvg,PFAvg,FGPAvg,3PPAvg,FTPAvg
0,2010,3102,49.5,10.5,17.0,5.4,0.5,12.1,16.4,13.8,35.762779,26.294418,72.439394
1,2010,3103,63.2,14.7,17.5,6.7,3.2,11.9,22.9,17.7,44.804018,32.864071,71.996477
2,2010,3104,56.3,10.6,20.0,6.9,3.3,10.9,26.2,19.1,39.052074,29.436109,62.765042
3,2010,3105,59.8,10.6,23.8,8.6,6.3,12.1,24.3,20.2,40.934253,34.251748,65.483965
4,2010,3106,57.1,13.2,21.8,7.9,5.5,15.4,25.2,21.3,39.069142,29.839286,65.277512


In [29]:
#X_train = np.zeros(shape=(n_test_games, 10))
X_train = []
y_train = []
# find end of season ELO ratings and regular season rolling averages for each tournament game played
for ii, row in df_concat.iterrows():
    win_team_features = []
    lose_team_features = []
    win_elo = row.WTeamELO
    lose_elo = row.LTeamELO
    win_team_features.append(win_elo)
    lose_team_features.append(lose_elo)
    # don't want to append the season and team ID here
    win_team_avgs = df_average_data[(df_average_data.Season == row.Season) & (df_average_data.TeamID == row.WTeamID)].iloc[0].values[2:]
    for average in win_team_avgs:
        win_team_features.append(average)
        
    lose_team_avgs = df_average_data[(df_average_data.Season == row.Season) & (df_average_data.TeamID == row.LTeamID)].iloc[0].values[2:]
    for average in lose_team_avgs:
        lose_team_features.append(average)
    
    # Randomly select win and lose order to train for both classes (0 and 1)
    if random.random() > 0.5:
        X_train.append(win_team_features + lose_team_features)
        y_train.append(1)
    else:
        X_train.append(lose_team_features + win_team_features)
        y_train.append(0)
        

IndexError: single positional indexer is out-of-bounds

In [ ]:
# Sanity check
print("X_train length is: " + str(len(X_train)))
print("y_train length is: " + str(len(y_train)))
print("First item in X_train vector is:")
X_train[0]

In [ ]:
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
len(X_train)

## Train the model ##
Use a basic logistic regression to train the model. You can set different C values to see how performance changes.

In [ ]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=25)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

## Format season and team IDs from the SampleSubmissionStage1.csv file ##


In [ ]:
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage2.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [ ]:
df_sample_sub.head()

In [ ]:
df_sample_sub.tail()

In [ ]:
# Confirm that the df_elo_ratings still has complete (1985-2018) data
df_elo_ratings.Season.sort_values().unique()

In [ ]:
# Rename team ID column back to the generic form
df_elo_ratings = df_elo_ratings.rename(columns={'LTeamID': 'TeamID'})

In [ ]:
df_elo_ratings.tail()

## Changed function from sample notebook to grab ELO Ratings and averaged stats##
Create predictions using the logistic regression model we trained.

In [ ]:
X_test = []
for ii, row in df_sample_sub.iterrows():
    team1_features = []
    team2_features = []
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_elo = df_elo_ratings[(df_elo_ratings.TeamID == t1) & (df_elo_ratings.Season == year)].season_elo.values[0]
    t1_avgs = df_average_data[(df_average_data.Season == year) & (df_average_data.TeamID == t1)].iloc[0].values[2:]
    t2_elo = df_elo_ratings[(df_elo_ratings.TeamID == t2) & (df_elo_ratings.Season == year)].season_elo.values[0]
    t2_avgs = df_average_data[(df_average_data.Season == year) & (df_average_data.TeamID == t2)].iloc[0].values[2:]
    team1_features.append(t1_elo)
    for average in t1_avgs:
        team1_features.append(average)
    team2_features.append(t2_elo)
    for average in t2_avgs:
        team2_features.append(average)
    X_test.append(team1_features + team2_features)

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [ ]:
len(X_test)

In [ ]:
preds = clf.predict_proba(X_test)[:,1]
df_sample_sub.Pred = preds
df_sample_sub.head()

Lastly, create your submission file!

In [ ]:
df_sample_sub.to_csv('dan_douthit_elo_recent_stats_2018.csv', index=False)